In [1]:
!pip install django pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 39.1 MB/s eta 0:00:00


In [2]:
import os

os.system("django-admin startproject restaurant_project")
os.chdir("restaurant_project")
os.system("python manage.py startapp restaurant")


0

In [3]:
settings_path = "restaurant_project/settings.py"

with open(settings_path, "r") as file:
    content = file.read()

content = content.replace(
    "INSTALLED_APPS = [",
    "INSTALLED_APPS = [\n    'restaurant',"
).replace(
    "ALLOWED_HOSTS = []",
    "ALLOWED_HOSTS = ['*']"
)

with open(settings_path, "w") as file:
    file.write(content)


In [4]:
model_code = '''
from django.db import models

class Table(models.Model):
    number = models.IntegerField(unique=True)
    capacity = models.IntegerField()
    is_reserved = models.BooleanField(default=False)

class MenuItem(models.Model):
    name = models.CharField(max_length=100)
    price = models.FloatField()
    stock = models.IntegerField()

class Order(models.Model):
    table = models.ForeignKey(Table, on_delete=models.CASCADE)
    items = models.ManyToManyField(MenuItem)
    timestamp = models.DateTimeField(auto_now_add=True)

class Reservation(models.Model):
    table = models.ForeignKey(Table, on_delete=models.CASCADE)
    name = models.CharField(max_length=100)
    time = models.DateTimeField()

class Inventory(models.Model):
    item_name = models.CharField(max_length=100)
    quantity = models.IntegerField()
'''
with open("restaurant/models.py", "w") as f:
    f.write(model_code)


In [5]:
!python manage.py makemigrations
!python manage.py migrate


Migrations for 'restaurant':
  restaurant/migrations/0001_initial.py
    + Create model Inventory
    + Create model MenuItem
    + Create model Table
    + Create model Reservation
    + Create model Order
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, restaurant, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length..

In [6]:
import django
import threading
import os

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'restaurant_project.settings')
django.setup()

def create_superuser():
    from django.contrib.auth.models import User
    if not User.objects.filter(username='admin').exists():
        User.objects.create_superuser('admin', 'admin@example.com', 'admin123')
        print("✅ Superuser created.")
    else:
        print("⚠️ Superuser already exists.")

thread = threading.Thread(target=create_superuser)
thread.start()
thread.join()


✅ Superuser created.


In [7]:
# restaurant/views.py
with open("restaurant/views.py", "w") as f:
    f.write('''
from django.http import JsonResponse
from .models import MenuItem, Table, Order, Inventory

def home(request):
    return JsonResponse({"message": "🍽️ Welcome to Restaurant API"})

def menu(request):
    items = list(MenuItem.objects.values())
    return JsonResponse(items, safe=False)
''')

# restaurant/urls.py
with open("restaurant/urls.py", "w") as f:
    f.write('''
from django.urls import path
from .views import home, menu

urlpatterns = [
    path('', home),
    path('menu/', menu),
]
''')

# restaurant_project/urls.py
with open("restaurant_project/urls.py", "w") as f:
    f.write('''
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('restaurant.urls')),
]
''')


In [12]:
# ✅ Kill any process using port 8000 (prevent port conflicts)
!kill -9 $(lsof -t -i:8000) 2>/dev/null

from pyngrok import ngrok
import os
import time
import threading

# 🔐 STEP 1: Replace with your working Ngrok Authtoken
# 👉 Get it from: https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("2zJ93Z39UdO1hWYmyihvX6sV21y_7k8RYnrPg3ifaMTATnekH")  # 🔁 Replace this string!

# 🚀 STEP 2: Start Django server in a thread
def run_server():
    os.system("python manage.py runserver 0.0.0.0:8000")

thread = threading.Thread(target=run_server)
thread.start()

# ⏳ STEP 3: Wait for server to boot up
time.sleep(5)

# 🌐 STEP 4: Create public ngrok tunnel
public_url = ngrok.connect(8000)
print(f"🚀 Django server is live at: {public_url}")


🚀 Django server is live at: NgrokTunnel: "https://d6f9-34-83-175-200.ngrok-free.app" -> "http://localhost:8000"
